In [1]:
# | default_exp tools.fs_read

In [ ]:
# | export
__all__ = ['logger', 'ToolCallMode', 'FsReadOperation', 'FsReadParams', 'FsReadTool']

# %% ../../nbs/buddy/backend/tools/filesystem/fs_read.ipynb 1
import os
import re
import time
import logging
from pathlib import Path
from typing import Dict, List, Any, Optional
import fnmatch
import json
from pydantic import BaseModel, field_validator, Field, ValidationInfo, ValidationError
from enum import Enum
from collections import defaultdict

# Set up logging (configurable level)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class ToolCallMode(str, Enum):
    DISCOVER = "discover"
    EXTRACT = "extract"

class FsReadOperation(BaseModel):
    mode: ToolCallMode
    path: str = Field(..., description="Specific file or directory path to operate on, e.g., 'src' or 'app.py'. Relative to project root if not absolute.")
    query: Optional[str] = Field(None, description="Search query as regex for DISCOVER (file name) or EXTRACT (content). Required for EXTRACT.")
    file_pattern: str = Field("*", description="Glob filter for files, e.g., '*.py' or '*.go|*.rs'.")
    max_depth: Optional[int] = Field(10, ge=1, description="Maximum recursion depth for DISCOVER mode. Value SHOULD BE >= 1")
    max_files: Optional[int] = Field(50, ge=1, description="Maximum number of files to return in DISCOVER mode")

    @field_validator("path")
    @classmethod
    def validate_path(cls, value: str, info: ValidationInfo) -> str:
        try:
            project_root = os.getcwd()
            if not value:
                value = project_root
                logger.info(f"Path empty; defaulting to project root: {value}")
            path_obj = Path(value).resolve()
            if not path_obj.exists():
                raise ValueError(f"Path {value} does not exist")
            if not str(path_obj).startswith(project_root):
                raise ValueError(f"Path {value} is outside project root; explicit permission is required to access external paths.")
            return str(path_obj)
        except Exception as e:
            logger.error(f"Path validation failed: {str(e)}")
            raise ValueError(f"Invalid path: {str(e)}")

    @field_validator("query")
    @classmethod
    def validate_query(cls, value: Any, info: ValidationInfo) -> Any:
        mode = info.data.get("mode")
        if mode == ToolCallMode.EXTRACT and value is None:
            raise ValueError("Query is required for extract mode to perform regex matching.")
        if mode == ToolCallMode.DISCOVER and value is None:
            return ""
        if value:
            try:
                re.compile(value)
            except re.error as e:
                logger.error(f"Invalid regex query: {value} ({str(e)})")
                raise ValueError(f"Invalid regex query: {value} ({str(e)})")
        return value

    @field_validator("file_pattern")
    @classmethod
    def validate_file_pattern(cls, value: Any) -> Any:
        if not isinstance(value, str):
            raise ValueError("file_pattern must be a string")
        try:
            for pattern in value.split('|'):
                fnmatch.fnmatch("test.txt", pattern)
        except Exception as e:
            logger.error(f"Invalid glob pattern: {value} ({str(e)})")
            raise ValueError(f"Invalid glob pattern: {value} ({str(e)})")
        return value

class FsReadParams(BaseModel):
    operations: List[FsReadOperation]

from agentic.tools.base import BaseTool, ToolMetadata, ToolCategory

class FsReadTool(BaseTool):
    def __init__(self, log_level: str = "INFO"):
        metadata = ToolMetadata(
            name="fs_read",
            description="Read filesystem with regex search and exclusions, supporting file discovery or content extraction",
            category=ToolCategory.FILESYSTEM
        )
        super().__init__(metadata)
        logging.getLogger().setLevel(getattr(logging, log_level, logging.INFO))
        self.project_root = os.getcwd()
        self.exclusion_patterns = [
            ".*", "*.pyc", "*.o", "*.obj", "*.class", "*.exe", "*.dll", "*.so",
            "*.lock", "node_modules/*", "dist/*", "build/*", "__pycache__/*",
            "*.bin", "*.zip", "*.tar.gz", "*.log"
        ]
        self._load_gitignore()

    def _load_gitignore(self) -> None:
        """Load .gitignore patterns to exclude files."""
        gitignore_path = Path(self.project_root) / ".gitignore"
        if gitignore_path.exists():
            try:
                with open(gitignore_path, 'r', encoding='utf-8', errors='ignore') as f:
                    for line in f:
                        line = line.strip()
                        if line and not line.startswith('#'):
                            self.exclusion_patterns.append(line)
            except (OSError, UnicodeDecodeError) as e:
                logger.debug(f"Failed to read {gitignore_path}: {e}")

    def _is_excluded(self, rel_path: str) -> bool:
        """Check if a path matches exclusion patterns."""
        return any(fnmatch.fnmatch(rel_path, p) for p in self.exclusion_patterns)

    def _get_exclusion_weight(self, rel_path: str) -> int:
        """Assign a weight to prioritize non-excluded files."""
        if rel_path.startswith('.'):
            return 5
        lower_path = rel_path.lower()
        if any(term in lower_path for term in ['venv', 'env', 'node_modules', 'dist', 'build']):
            return 1
        if any(term in lower_path for term in ['__pycache__', '.cache', 'temp']):
            return 2
        return 3  # Default for other excluded

    def _get_file_info(self, path: str) -> Dict[str, Any]:
        """Get metadata about a file, including size and binary status."""
        path_obj = Path(path)
        if not path_obj.exists():
            return {"error": "File not found"}

        try:
            stat = path_obj.stat()
        except OSError as e:
            return {"error": f"Stat failed: {e}"}

        rel_path = os.path.relpath(path, self.project_root)
        if self._is_excluded(rel_path):
            return {"error": "Excluded by patterns"}

        file_size = stat.st_size
        is_binary = False
        lines = 0

        try:
            with open(path, 'rb') as f:
                chunk = f.read(2048)  # Larger sample for better binary detection
                if b'\0' in chunk:
                    is_binary = True
                else:
                    try:
                        chunk.decode('utf-8')
                        with open(path, 'r', encoding='utf-8', errors='ignore') as f_text:
                            lines = sum(1 for _ in f_text)
                    except UnicodeDecodeError:
                        is_binary = True
        except (OSError, UnicodeDecodeError) as e:
            logger.debug(f"Error checking file {path}: {str(e)}")
            is_binary = True

        return {
            "size": file_size,
            "is_binary": is_binary,
            "file_type": path_obj.suffix.lower(),
            "is_large": file_size > 1024 * 1024,
            "lines": lines if not is_binary else 0
        }

    def _build_tree(self, rel_paths: List[str]) -> str:
        """Build a tree representation of file paths."""
        tree = {}
        for rel_path in rel_paths:
            parts = rel_path.split(os.sep)
            current = tree
            for part in parts[:-1]:  # Dirs
                if part not in current:
                    current[part] = {}
                current = current[part]
            if parts:  # File
                current[parts[-1]] = {}  # Empty dict for file

        def print_tree(node, prefix: str = "") -> List[str]:
            lines = []
            items = sorted(node.keys())
            for i, item in enumerate(items):
                is_last = i == len(items) - 1
                connector = "└── " if is_last else "├── "
                lines.append(prefix + connector + item)
                sub_prefix = prefix + ("    " if is_last else "│   ")
                lines.extend(print_tree(node[item], sub_prefix))
            return lines

        tree_lines = print_tree(tree)
        return "\n".join(tree_lines) if tree_lines else "No files found"

    def _discover_files(self, path: str, file_pattern: str, query: str, max_depth: int, max_files: int) -> str:
        """Discover files matching patterns and query, returning a tree structure."""
        start_time = time.time()
        path_obj = Path(path)
        if not path_obj.is_dir():
            return json.dumps({"error": "Path is not a directory for discovery"})

        query_pattern = re.compile(query, re.IGNORECASE) if query else None
        patterns = [fnmatch.translate(p) for p in file_pattern.split('|')]
        pattern_regexes = [re.compile(p) for p in patterns]
        candidates = []

        def _collect_with_walk(dir_path: str, current_depth: int, file_count: List[int]) -> None:
            if current_depth > max_depth or file_count[0] >= max_files:
                return
            try:
                for entry in os.scandir(dir_path):
                    rel_path = entry.name
                    try:
                        rel_path = str(Path(entry.path).relative_to(self.project_root))
                    except ValueError:
                        pass
                    if self._is_excluded(rel_path):
                        continue
                    if entry.is_file() and any(p.match(entry.name) for p in pattern_regexes):
                        if query_pattern and not query_pattern.search(entry.name):
                            continue
                        weight = 10 if not self._is_excluded(rel_path) else self._get_exclusion_weight(rel_path)
                        candidates.append((entry.path, weight))
                        file_count[0] += 1
                    if entry.is_dir():
                        _collect_with_walk(entry.path, current_depth + 1, file_count)
            except (OSError, PermissionError) as e:
                logger.debug(f"Scan error in {dir_path}: {e}")

        file_count = [0]
        _collect_with_walk(str(path_obj), 0, file_count)

        if not candidates:
            return json.dumps({"error": "No files matched the criteria"})

        candidates.sort(key=lambda x: (-x[1], x[0]))
        selected_paths = [p for p, w in candidates[:max_files]]
        rel_paths = [os.path.relpath(p, path) for p in selected_paths]
        tree_str = self._build_tree(rel_paths)

        logger.info(f"Discovered {len(selected_paths)} files in {time.time() - start_time:.2f}s")
        return json.dumps({"data": tree_str})

    def _extract_content(self, path: str, query: str, file_pattern: str) -> str:
        """Extract content from files matching query and pattern."""
        path_obj = Path(path)
        if not path_obj.exists():
            return json.dumps({"error": f"Path {path} does not exist"})

        if path_obj.is_file():
            rel_path = os.path.relpath(path, self.project_root)
            if self._is_excluded(rel_path):
                return json.dumps({"error": f"Path {path} is excluded by patterns"})
            content = self._extract_from_file(path, query)
            return json.dumps({"data": content}, ensure_ascii=False)
        else:
            snippets = []
            for root, dirs, files in os.walk(path):
                for file in files:
                    if any(fnmatch.fnmatch(file, p) for p in file_pattern.split('|')):
                        full_path = os.path.join(root, file)
                        rel_path = os.path.relpath(full_path, self.project_root)
                        if not self._is_excluded(rel_path):
                            file_snip = self._extract_from_file(full_path, query)
                            if file_snip and not file_snip.startswith("[Binary") and not file_snip.startswith("Error"):
                                snippets.append({"file": rel_path, "snippet": file_snip[:1000]})  # Limit snippet size
            if not snippets:
                return json.dumps({"error": f"No valid text files matched the criteria in {path}"})
            try:
                content = json.dumps({"data": snippets}, ensure_ascii=False)
                if len(content) > 16 * 1024:
                    # Truncate snippets instead of the JSON
                    truncated_snippets = snippets[:10]  # Limit to first 10 files
                    content = json.dumps({"data": truncated_snippets, "truncated": True}, ensure_ascii=False)
                return content
            except Exception as e:
                return json.dumps({"error": f"JSON serialization failed: {str(e)}"})

    def _extract_from_file(self, file_path: str, query: str) -> str:
        """Extract content from a single file with regex matching."""
        file_info = self._get_file_info(file_path)
        if "error" in file_info:
            return f"Error: {file_info['error']}"
        if file_info["is_binary"]:
            return f"[Binary file: {file_info['size']} bytes]"
        try:
            pattern = re.compile(query, re.IGNORECASE | re.MULTILINE) if query else None
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                lines = f.readlines()
            if not query:
                content = "".join(lines)
                if len(content) > 1024:  # Smaller limit
                    content = content[:1024] + "... [truncated]"
                return content + f"\n--- File Info: {file_info['lines']} lines, {file_info['size']} bytes ---"
            matches = []
            for line_num, line in enumerate(lines, 1):
                if pattern and pattern.search(line):
                    # Clean the line to avoid JSON issues
                    clean_line = line.strip().replace('"', '\\"').replace('\n', '\\n').replace('\r', '\\r')
                    matches.append(f"Line {line_num}: {clean_line}")
                    if len(matches) >= 20:  # Limit matches per file
                        break
            if not matches:
                return "No matches found"
            content = "\n".join(matches)
            if len(content) > 16 * 1024:
                content = content[:16 * 1024] + "... [truncated]"
            return content + f"\n--- File Info: {file_info['lines']} lines, {file_info['size']} bytes ---"
        except (OSError, UnicodeDecodeError) as e:
            logger.error(f"Error reading file {file_path}: {str(e)}")
            return f"Error reading file: {str(e)}"

    def get_parameters_schema(self, verbose: bool = True) -> Dict[str, Any]:
        """Return OpenAI-compatible schema for the tool."""
        schema = {
            "type": "function",
            "function": {
                "name": "fs_read",
                "description": "Discover files in tree structure or extract regex-matched snippets. Uses exclusions with fallback sorting." if verbose else "",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "operations": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "mode": {
                                        "type": "string",
                                        "enum": [mode.value for mode in ToolCallMode],
                                        "description": "Select 'discover' to list files in tree or 'extract' to pull snippets." if verbose else ""
                                    },
                                    "path": {
                                        "type": "string",
                                        "description": "File or directory path relative to project root, e.g., 'src' or 'app.py'." if verbose else ""
                                    },
                                    "query": {
                                        "type": "string",
                                        "description": "Regex for fuzzy file name (DISCOVER) or content (EXTRACT). Required for EXTRACT." if verbose else ""
                                    },
                                    "file_pattern": {
                                        "type": "string",
                                        "description": "Glob filter, e.g., '*.py' or '*.go|*.rs'." if verbose else ""
                                    },
                                    "max_depth": {
                                        "type": "integer",
                                        "description": "Maximum recursion depth for DISCOVER mode (default: 10)." if verbose else ""
                                    },
                                    "max_files": {
                                        "type": "integer",
                                        "description": "Maximum number of files to return in DISCOVER mode (default: 50)." if verbose else ""
                                    }
                                },
                                "required": ["mode", "path"]
                            }
                        }
                    },
                    "required": ["operations"]
                }
            }
        }
        return schema["function"]["parameters"]

    def execute(self, **kwargs) -> Dict[str, Any]:
        """Execute filesystem read operations with robust error handling."""
        try:
            if 'params' in kwargs:
                params = kwargs['params']
            else:
                if 'operations' not in kwargs and ('mode' in kwargs or 'path' in kwargs):
                    kwargs = {'operations': [kwargs]}
                params = FsReadParams(**kwargs)
            result = self._execute_internal(params)
            # Simplify response for single successful operation
            if len(params.operations) == 1 and result["success"] and not result["error"]:
                op_result = result["data"][0]
                return {
                    "success": True,
                    "data": json.loads(op_result["data"])["data"] if op_result["data"] else None,
                    "error": None,
                    "metadata": {"processed_files": len(result["data"])}
                }
            return result
        except ValidationError as e:
            logger.error(f"Input validation failed: {str(e)}")
            return {"success": False, "data": [], "error": {"type": "ValidationError", "message": str(e)}, "metadata": {}}
        except AttributeError as e:
            logger.error(f"Attribute error in execution: {str(e)}")
            return {"success": False, "data": [], "error": {"type": "AttributeError", "message": str(e)}, "metadata": {}}
        except Exception as e:
            logger.critical(f"Unexpected execution error: {str(e)}")
            return {"success": False, "data": [], "error": {"type": "UnexpectedError", "message": str(e)}, "metadata": {}}

    def _execute_internal(self, params: FsReadParams) -> Dict[str, Any]:
        """Internal execution logic for batch operations."""
        results = []
        total_processed = 0

        for op in params.operations:
            mode = op.mode.value
            path = op.path
            output_data = None
            error = None

            try:
                if mode == "discover":
                    output_data = self._discover_files(path, op.file_pattern, op.query or "", op.max_depth, op.max_files)
                    total_processed += op.max_files  # Approximate
                elif mode == "extract":
                    output_data = self._extract_content(path, op.query, op.file_pattern)
                    total_processed += 1
            except ValueError as e:
                if "outside project root" in str(e):
                    error = {"type": "PermissionError", "message": str(e)}
                else:
                    error = {"type": "ValueError", "message": str(e)}
                logger.error(f"Error in {mode} on {path}: {str(e)}")
            except (OSError, UnicodeDecodeError, re.error) as e:
                error = {"type": type(e).__name__, "message": str(e)}
                logger.error(f"Error in {mode} on {path}: {str(e)}")

            results.append({
                "mode": mode,
                "path": path,
                "data": output_data,
                "error": error
            })

        overall_success = all(r.get("error") is None for r in results)
        overall_error = None if overall_success else {"type": "BatchError", "message": "Some operations failed—check individual errors."}
        return {
            "success": overall_success,
            "data": results,
            "error": overall_error,
            "metadata": {"processed_files": total_processed}
        }


ImportError: attempted relative import with no known parent package